# **Deployment, Monitoring & Continuous Improvement**

**✨ Cross‑Environment Compatibility Statement (Polished Version)**

This hands‑on demo was built and tested in Microsoft Fabric, but every step uses standard Apache Spark APIs. That means you can run the exact same code in any Spark‑compatible environment, including Databricks, Synapse Spark, AWS EMR, Google Dataproc, or a local Spark cluster. No Fabric‑specific libraries or features are required.

As long as your environment supports:
- Apache Spark (PySpark)
- Standard DataFrame operations
- Basic file I/O to a supported storage location

In [1]:
# Replace the path with your file location in the Lakehouse:
csv_path = "Files/csv/ml/testing/regression_housing_testing.csv"

# Read with header, infer schema; we'll fix types afterwards to be safe.
df_test_raw_data = (
    spark.read
         .option("header", "true")
         .option("inferSchema", "true")
         # Treat empty strings as nulls to simplify downstream casting
         .option("nullValue", "")
         .csv(csv_path)
)

display(df_test_raw_data.limit(5))

SynapseWidget(Synapse.DataFrame, 11c80ee9-7d00-49b2-8ab2-f8f22c24ade8)

In [2]:
from pyspark.sql import functions as F

# Columns to impute
target_cols = ["sqft", "lot_acres", "baths"]

# 1) Cast target columns to numeric (DoubleType) if needed
df_num = df_test_raw_data.select([
    F.col(c).cast("double").alias(c) if c in target_cols else F.col(c)
    for c in df_test_raw_data.columns
])

# 2) Compute medians using approxQuantile
#    relativeError=0.001 gives a tight approximation while staying efficient
medians = {}
for c in target_cols:
    # Handle the case where column is all nulls (approxQuantile returns empty)
    quantiles = df_num.stat.approxQuantile(c, [0.5], 0.001)
    medians[c] = quantiles[0] if quantiles else None

# Optional: if a column has no non-null values, fall back to 0 or mean
fallbacks = {}
for c in target_cols:
    if medians[c] is None:
        # fallback to mean; if mean is also None, fallback to 0
        mean_val = df_num.select(F.mean(F.col(c))).first()[0]
        fallbacks[c] = mean_val if mean_val is not None else 0.0

# Combine medians with fallbacks
fill_values = {c: (medians[c] if medians[c] is not None else fallbacks[c]) for c in target_cols}

# 3) Impute nulls
df_imputed = df_num.na.fill(fill_values)

# Preview results
display(df_imputed.limit(5))  # quick sanity check

StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1c759b5f-7454-4e3a-88a5-bfcaeb3bbf0c)

Your Model Is Only as Good as Its Preprocessing

In [3]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

cat_cols = ["city", "neighborhood"]

# Convert only the needed columns to pandas to avoid memory blowups
pdf_cat = df_imputed.select(cat_cols).toPandas()

enc = OneHotEncoder(handle_unknown="ignore", sparse_output=False)  # use sparse_output=True for large data
X_cat = enc.fit_transform(pdf_cat)
oh_cols = enc.get_feature_names_out(cat_cols)

# If you want the full encoded pandas DataFrame:
pdf_full = df_imputed.toPandas()
df_oh = pd.concat(
    [pdf_full.drop(columns=cat_cols).reset_index(drop=True),
     pd.DataFrame(X_cat, columns=oh_cols)],
    axis=1
)

print("Original columns:", len(pdf_full.columns))
print("One-hot encoded columns:", len(df_oh.columns))

display(df_oh.head(5))

StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 5, Finished, Available, Finished)

Original columns: 9
One-hot encoded columns: 20


SynapseWidget(Synapse.DataFrame, 2ab77d0e-d7e4-4ed3-bb69-297b8315444e)

In [4]:
import numpy as np
from sklearn.preprocessing import StandardScaler

target_col = "price"  # label
feature_cols = [c for c in df_oh.columns if c != target_col]

# Numeric columns to scale (skip one-hot 0/1 by nunique==2)
num_cols = [
    c for c in feature_cols
    if np.issubdtype(df_oh[c].dtype, np.number) and df_oh[c].nunique() > 2
]
oh_cols = [c for c in feature_cols if c not in num_cols]  # keep as-is

# Fit scaler on TRAIN ONLY in a real pipeline; for demo we fit on all
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(df_oh[num_cols])

# Create the final scaled dataframe
df_scaled = df_oh.copy()
df_scaled[num_cols] = scaler.transform(df_scaled[num_cols])

display(df_scaled.head(5))

StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4886f3d5-a15d-4f4d-a510-7ae7dda98b5c)

In [5]:
import mlflow

model_name = "diabetes-model-community-demo"
explicit_version = 2   # 👈 explicitly specify your version here

model_uri = f"models:/{model_name}/{explicit_version}"
print(f"Loading model from: {model_uri}")

loaded_model = mlflow.pyfunc.load_model(model_uri)


StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 7, Finished, Available, Finished)

Loading model from: models:/diabetes-model-community-demo/2


StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 8, Finished, Available, Finished)

StatementMeta(, , -1, Finished, , Finished)

RejectSilentExecuteRequest: Livy session has failed. Error code: RejectSilentExecuteRequest. Rejected silent execute_request as there is no active session.

StatementMeta(, , -1, Finished, , Finished)

RejectSilentExecuteRequest: Livy session has failed. Error code: RejectSilentExecuteRequest. Rejected silent execute_request as there is no active session.

In [7]:

# Assuming you already have:
# - df_oh (original one-hot dataframe)
# - df_scaled (scaled numeric columns)
# - feature_cols (all features excluding target_col)
# - loaded_model (mlflow.pyfunc model)

X = df_scaled[feature_cols]


StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 10, Finished, Available, Finished)

⚠️ **Feature alignment tip:** If the model has a stored input signature, use it to strictly select/arrange columns to prevent mismatch:

In [8]:

import mlflow

X = df_scaled[feature_cols]

try:
    info = mlflow.models.get_model_info(model_uri)  # optional; may vary with MLflow version
    if info.signature and info.signature.inputs:
        model_input_cols = [inp.name for inp in info.signature.inputs]
        X = df_scaled[model_input_cols]
    else:
        model_input_cols = feature_cols
except Exception:
    model_input_cols = feature_cols

StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 11, Finished, Available, Finished)

StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 12, Finished, Available, Finished)

StatementMeta(, , -1, Finished, , Finished)

RejectSilentExecuteRequest: Livy session has failed. Error code: RejectSilentExecuteRequest. Rejected silent execute_request as there is no active session.

StatementMeta(, , -1, Finished, , Finished)

RejectSilentExecuteRequest: Livy session has failed. Error code: RejectSilentExecuteRequest. Rejected silent execute_request as there is no active session.

#### **Batch predict + make results human‑readable (Pandas path)**
- This helper converts most common model outputs into readable columns:

- Regression → numeric prediction column + a formatted string (e.g., 1,234.56)
- Classification (labels) → integer class id + optional mapped label
- Multiclass probabilities → per-class p(class) columns + top class and confidence%

In [9]:

import numpy as np
import pandas as pd

# 1) Ensure index alignment between features and predictions
X = df_scaled[feature_cols].copy()
X = X.reset_index(drop=True)
df_oh_aligned = df_oh.reset_index(drop=True)

# 2) Human-readable conversion helper (same as before, but returns columns with a prefix to avoid collisions)
def to_human_readable(raw_pred, label_map=None, prefix="scored_"):
    arr = np.asarray(raw_pred)
    n = arr.shape[0]

    # Detect output type
    if arr.ndim == 2 and arr.shape[1] > 1 and np.all((arr >= 0) & (arr <= 1)):
        task = 'multiclass_proba'
    elif arr.ndim == 2 and arr.shape[1] == 1:
        task = 'regression'
        arr = arr.reshape(-1)
    elif arr.ndim == 1 and np.issubdtype(arr.dtype, np.floating):
        task = 'regression'
    elif arr.ndim == 1 and np.issubdtype(arr.dtype, np.integer):
        task = 'classification_label'
    else:
        task = 'unknown'

    out = pd.DataFrame(index=np.arange(n))

    if task == 'regression':
        out[f'{prefix}prediction'] = arr.astype(float).round(2)
        out[f'{prefix}prediction_readable'] = out[f'{prefix}prediction'].map(lambda x: f"{x:,.2f}")
    elif task == 'classification_label':
        out[f'{prefix}predicted_class_id'] = arr.astype(int)
        if label_map:
            out[f'{prefix}predicted_class'] = out[f'{prefix}predicted_class_id'].map(label_map).fillna(
                out[f'{prefix}predicted_class_id'].astype(str)
            )
        else:
            out[f'{prefix}predicted_class'] = out[f'{prefix}predicted_class_id'].astype(str)
    elif task == 'multiclass_proba':
        classes = list(range(arr.shape[1]))
        prob_df = pd.DataFrame(arr, columns=[f"{prefix}p(class_{c})" for c in classes]).round(4)
        out = pd.concat([out, prob_df], axis=1)
        best_idx = np.argmax(arr, axis=1)
        best_prob = arr[np.arange(n), best_idx]
        out[f'{prefix}predicted_class_id'] = best_idx
        out[f'{prefix}confidence_%'] = (best_prob * 100).round(1)
        if label_map:
            out[f'{prefix}predicted_class'] = out[f'{prefix}predicted_class_id'].map(label_map).fillna(
                out[f'{prefix}predicted_class_id'].astype(str)
            )
        else:
            out[f'{prefix}predicted_class'] = out[f'{prefix}predicted_class_id'].astype(str)
    else:
        out[f'{prefix}raw_prediction'] = list(raw_pred)

    return out

# 3) Predict
raw_pred = loaded_model.predict(X)

# If classification, provide human-friendly labels:
# label_map = {0: "No Diabetes", 1: "Diabetes"}
label_map = None

human_preds = to_human_readable(raw_pred, label_map=label_map, prefix="scored_")

# 4) Concatenate and make sure column names are unique
df_scored = pd.concat([df_oh_aligned, human_preds], axis=1)

# If any duplicates still exist (e.g., df_oh had a "scored_prediction" already), enforce uniqueness:
if not df_scored.columns.is_unique:
    # add numeric suffixes to duplicates
    df_scored.columns = pd.io.parsers.ParserBase({'names': df_scored.columns})._maybe_dedup_names(df_scored.columns)

# 5) Preview safely
display(df_scored.head(10))

StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7b5f60ae-387f-4cc5-acc3-ee55c940a8a3)

In [10]:

# If your regression target is price and you want $ formatting:
df_scored["scored_prediction_usd"] = df_scored["scored_prediction"].map(lambda x: f"${x:,.2f}")
display(df_scored[["scored_prediction", "scored_prediction_usd"]].head(10))


StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 887e0171-7eb6-4c87-8122-406daee98895)

Create a deterministic row_id for the current dataset snapshot

In [11]:

# 1) Add a stable row_id to df_imputed (Spark)
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id


# Create a deterministic row_id for the current dataset snapshot
w = Window.orderBy(monotonically_increasing_id())
df_imputed_with_id = df_imputed.withColumn("row_id", row_number().over(w))

# 2) Bring df_imputed_with_id to Pandas for one-hot/scaling if your transforms are Pandas-based
pdf_imputed = df_imputed_with_id.toPandas()




StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 15, Finished, Available, Finished)

In [12]:

# 1) Add a stable row_id to df_imputed (Spark)
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id


# Create a deterministic row_id for the current dataset snapshot
w = Window.orderBy(monotonically_increasing_id())
df_imputed_with_id = df_imputed.withColumn("row_id", row_number().over(w))

# 2) Bring df_imputed_with_id to Pandas for one-hot/scaling if your transforms are Pandas-based
pdf_imputed = df_imputed_with_id.toPandas()

# After you compute human_preds and df_scored:
# human_preds = to_human_readable(...)
# df_scored = pd.concat([df_oh_aligned, human_preds], axis=1)

# 🔒 Ensure row_id is present in df_scored
if "row_id" not in df_scored.columns:
    # Use the row_id we propagated from df_imputed earlier
    # pdf_imputed is the Pandas version of df_imputed_with_id (Spark) that has row_id
    df_scored["row_id"] = pdf_imputed["row_id"].values

# Format USD
df_scored["scored_prediction_usd"] = df_scored["scored_prediction"].map(lambda x: f"${x:,.2f}")

# Now this will work:
sdf_preds = spark.createDataFrame(df_scored[["row_id", "scored_prediction", "scored_prediction_usd"]])

df_imputed_scored = df_imputed_with_id.join(sdf_preds, on="row_id", how="left")

# Optional: keep preds at the end
base_cols = [c for c in df_imputed.columns]
df_imputed_scored = df_imputed_scored.select(*base_cols, "scored_prediction", "scored_prediction_usd")

display(df_imputed_scored.limit(10))


StatementMeta(, fd03863d-c671-4277-abf5-34d74664a3c5, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 46c8e353-0c92-4135-be57-f17821622cd4)

Persist results to your Microsoft Fabric Lakehouse

In [ ]:

# Overwrite or append depending on your pipeline needs
df_imputed_scored.write.mode("overwrite").saveAsTable("diabetes_scored")  # choose a table name

# Verify
display(spark.table("diabetes_scored").limit(20))


end good

In [ ]:
import mlflow.pyfunc

model = mlflow.pyfunc.load_model("models:/diabetes-model-community-demo/2")
predictions = model.predict(your_dataframe)
